In [1]:
import cv2
import numpy as np
import os
from IPython.display import Video, display
import matplotlib.pyplot as plt

In [3]:
input_img  = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Input Images\solidWhiteCurve.jpg"
output_img = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Output.jpg"

In [5]:
input_video  = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Input Video\solidWhiteRight.mp4"
output_video = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Output.mp4"

In [7]:
#Edge Detection
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges

In [9]:
#ROI
def region_of_interest(image):
    h, w = image.shape[:2]
    polygon = np.array([[(0,h), (w,h), (w//2, int(h*0.55))]])
    
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(image, mask)

In [11]:
#Line Detection + Averaging
def make_coordinates(image, line):
    slope, intercept = line
    h = image.shape[0]
    y1 = h
    y2 = int(h*0.6)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1,y1,x2,y2])

In [13]:
#Extract and Average Hough lines
def average_slope_intercept(image, lines):
    if lines is None:
        return []

    left = []
    right = []

    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        slope, intercept = np.polyfit((x1,x2),(y1,y2),1)

        if slope < 0:
            left.append((slope,intercept))
        else:
            right.append((slope,intercept))

    final = []

    if len(left):
        left_avg = np.average(left, axis=0)
        final.append(make_coordinates(image, left_avg))
    if len(right):
        right_avg = np.average(right, axis=0)
        final.append(make_coordinates(image, right_avg))

    return final


In [15]:
#Lines Display
def display_lines(image, lines):
    line_img = np.zeros_like(image)

    if lines is None or len(lines)==0:
        return line_img

    for line in lines:
        if line is None: continue
        x1,y1,x2,y2 = line
        cv2.line(line_img,(x1,y1),(x2,y2),(0,255,0),8)

    return line_img

In [17]:
#lane Detection Pipeline
def pipeline(image):
    edges = canny(image)
    roi = region_of_interest(edges)

    lines = cv2.HoughLinesP(roi, 2, np.pi/180, 20, np.array([]), minLineLength=20, maxLineGap=50)

    averaged_lines = average_slope_intercept(image, lines)
    line_img = display_lines(image, averaged_lines)
    combo = cv2.addWeighted(image, 0.8, line_img, 1, 1)
    return combo

In [19]:
import os
print("PB exists:", os.path.exists(r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\frozen_inference_graph.pb"))
print("PBTXT exists:", os.path.exists(r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"))
print("COCO.NAMES exists:", os.path.exists(r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\coco.names"))

PB exists: True
PBTXT exists: True
COCO.NAMES exists: True


In [21]:
#Load COCO labels
import cv2

model = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\frozen_inference_graph.pb"
config = r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"

net = cv2.dnn_DetectionModel(model, config)
net.setInputSize(320, 320)
net.setInputScale(1.0/127.5)
net.setInputMean((127.5,127.5,127.5))
net.setInputSwapRB(True)

with open(r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Models\coco.names") as f:
    classNames = f.read().split("\n")

In [23]:
print(len(classNames), classNames[:10])

81 ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light']


In [25]:
img = cv2.imread(r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Input Images\solidWhiteCurve.jpg")

classIds, confs, boxes = net.detect(img, confThreshold=0.3)
print("classIds:", classIds)
print("confs:", confs)
print("boxes:", boxes)

classIds: [ 3  3  3  3  3  3  3  3  3  3  3  8  3  3  8  3 10  3  8  3  8  3  8  8
  3  3]
confs: [0.75402963 0.71150243 0.60353255 0.580633   0.5750482  0.52799016
 0.52189314 0.4865406  0.4548775  0.45088458 0.43860966 0.43652895
 0.41465262 0.3836329  0.3828955  0.3760134  0.37314135 0.3686097
 0.3611433  0.3601551  0.35467285 0.3283515  0.32755202 0.3261915
 0.31044134 0.30146742]
boxes: [[185 305  83  47]
 [104 307  69  45]
 [ 56 305  28  15]
 [281 302  39  32]
 [425 302  36  20]
 [393 305  30  20]
 [488 301  27  15]
 [366 304  26  18]
 [104 307  44  19]
 [195 302  64  37]
 [255 310  42  38]
 [185 305  83  47]
 [263 304  35  28]
 [533 289  30  19]
 [281 302  39  32]
 [144 304  37  24]
 [293 280  14  14]
 [574 277  22  22]
 [104 307  69  45]
 [281 314  32  24]
 [425 302  36  20]
 [ 62 309  27  18]
 [393 305  30  20]
 [488 301  27  15]
 [ 36 311  25  12]
 [567 280  24  20]]


In [27]:
#Image Detection
def detect_objects(image, conf=0.6):
    classIds, confs, boxes = net.detect(image, confThreshold=conf)
    objs = []

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), boxes):
            
            label = classNames[classId-1]

            # ✅ Filter only vehicles
            if label not in ["car", "truck", "bus"]:
                continue

            objs.append((label, float(confidence), box))

    return objs

In [29]:
#Single Image Drawing Boxes
def draw_objects(image, objects):
    for label, confidence, box in objects:
        x, y, w, h = map(int, box)

        cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(image,
                    f"{label} {confidence:.2f}",
                    (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0,255,0), 2)

    return image

In [31]:
#Run on Single Image
image = cv2.imread(input_img)

objects = detect_objects(image)      # detect
result  = draw_objects(image, objects)   # annotate

cv2.imwrite(output_img, result)
print(" Saved output to:", output_img)

 Saved output to: C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Output.jpg


In [33]:
frame = cv2.imread(r"C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Input Images\solidWhiteCurve.jpg")  
objects = detect_objects(frame)
print(objects)

[('car', 0.7540296316146851, array([185, 305,  83,  47], dtype=int32)), ('car', 0.7115024328231812, array([104, 307,  69,  45], dtype=int32)), ('car', 0.6035325527191162, array([ 56, 305,  28,  15], dtype=int32))]


In [35]:
#Video Object Detection Function
def detect_objects(image, conf=0.6):
    classIds, confs, boxes = net.detect(image, confThreshold=conf)
    objs = []

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), boxes):
            label = classNames[classId - 1]
            if label not in ["car", "truck", "bus"]:
                continue
            objs.append((label, float(confidence), box))
    return objs

In [37]:
#Video Drawing Objects
def draw_objects(image, objects):
    for label, confidence, box in objects:
        x, y, w, h = map(int, box)
        cv2.rectangle(image, (x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(image, f"{label} {confidence:.2f}",
                    (x,y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,(0,255,0),2)
    return image

In [39]:
#Video Perception 
def perception_pipeline(frame):
    lane_img = pipeline(frame)
    objects = detect_objects(frame)
    final = annotate_objects(lane_img, objects)
    return final

In [41]:
#Run and Saving Video 
cap = cv2.VideoCapture(input_video)

w = int(cap.get(3))
h = int(cap.get(4))
fps = cap.get(5) if cap.get(5) > 0 else 30

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (w,h))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    objs = detect_objects(frame)
    frame = draw_objects(frame, objs)   

    out.write(frame)

cap.release()
out.release()
print("Video saved:", output_video)

Video saved: C:\Users\Jahnavi\Desktop\Internship_3 Sem\AI Self Driving Car\Basic Perception\Output.mp4
